<a href="https://colab.research.google.com/github/TheProgrammingArchive/EmotionDetection/blob/main/Fer%2BEmotionRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255, 
                                   zoom_range = 0.3,
                                   horizontal_flip=True)

train_data = train_datagen.flow_from_directory('/content/data/content/train/', batch_size=64, target_size=(48, 48), shuffle=True, color_mode='grayscale', class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1/255)

test_data = test_datagen.flow_from_directory('/content/data/content/test/', batch_size=64, target_size=(48, 48), shuffle=True, color_mode='grayscale', class_mode='categorical')


Found 30259 images belonging to 7 classes.
Found 4887 images belonging to 7 classes.


In [ ]:
train_data.class_indices

{'Angry': 0,
 'Disgust': 1,
 'Fear': 2,
 'Happy': 3,
 'Neutral': 4,
 'Sad': 5,
 'Surprise': 6}

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(7, activation='softmax'),
])

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 batch_normalization (BatchN  (None, 44, 44, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       7

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/Untitled Folder',
    save_best_only=True,
    mode='max',
    monitor='val_acc'
)

earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    restore_best_weights=True,
    min_delta=0
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    patience=7,
    verbose=1,
    min_delta=0.0001,
    factor=0.2
)

callbacks_ = [checkpoint, earlystop, reduce_lr]

In [ ]:
model.fit(train_data, validation_data=test_data, epochs=50, callbacks=callbacks_, steps_per_epoch=train_data.n//train_data.batch_size, validation_steps=test_data.n//test_data.batch_size)

472/472 [==============================] - ETA: 0s - loss: 0.8505 - accuracy: 0.6850

472/472 [==============================] - 110s 232ms/step - loss: 0.8505 - accuracy: 0.6850 - val_loss: 1.0436 - val_accuracy: 0.6324 - lr: 4.0000e-06


In [ ]:
model.save('/content/model', save_format='h5')